In [1]:
import fitz  # PyMuPDF
import csv
from typing import List, Dict, Any, Tuple, Optional
from pathlib import Path

def extract_links_with_dimensions(pdf_path: Path, output_csv_path: Path):
    """
    從 PDF 中提取超連結，並包含頁面尺寸以便進行正規化。
    """
    doc = fitz.open(pdf_path)
    links_data = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_width = page.rect.width
        page_height = page.rect.height

        for link in page.get_links():
            uri = link.get("uri")
            if not uri:
                continue
            
            rect = link.get("from")
            if not rect:
                continue

            links_data.append({
                "page": page_num,  # 使用 0-based 索引，與 MinerU 對齊
                "page_width": page_width,
                "page_height": page_height,
                "uri": uri,
                "rect": str(rect),
            })

    doc.close()

    if not links_data:
        print(f"⚠️ 在 {pdf_path.name} 中沒有找到任何超連結。")
        return

    # 寫入 CSV
    output_csv_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_csv_path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=links_data[0].keys())
        writer.writeheader()
        writer.writerows(links_data)

    print(f"✅ 從 {pdf_path.name} 中提取了 {len(links_data)} 個超連結。")
    print(f"📁 輸出已儲存至 {output_csv_path}")


DOC_CODE = "AML-1"
PDF_PATH = Path(f"c:/Users/User/Desktop/GraphRAG/policy_dataset/IC-1 Risk Management Framework/{DOC_CODE}.pdf")
OUTPUT_LINKS_CSV = Path(f"c:/Users/User/Desktop/GraphRAG/Main_Workflow/Hyperlink_extract/{DOC_CODE}.links_normalized.csv")
    
extract_links_with_dimensions(PDF_PATH, OUTPUT_LINKS_CSV)

✅ 從 AML-1.pdf 中提取了 6 個超連結。
📁 輸出已儲存至 c:\Users\User\Desktop\GraphRAG\Main_Workflow\Hyperlink_extract\AML-1.links_normalized.csv


# Hyperlink Matching

In [2]:
import json, re, pandas as pd
from pathlib import Path
from collections import defaultdict

# ------------------------------------------------------------
# 0) 路徑
# ------------------------------------------------------------
# DOC_CODE = "IC-1"
CONTENT_LIST_JSON = Path(f"C:/Users/User/Desktop/GraphRAG/out_md_mineru/{DOC_CODE}/auto/{DOC_CODE}_content_list.json")
INPUT_LINKS_CSV   = Path(f"C:/Users/User/Desktop/GraphRAG/Main_Workflow/Hyperlink_extract/{DOC_CODE}.links_normalized.csv")
OUTPUT_MATCH_CSV  = Path(f"C:/Users/User/Desktop/GraphRAG/Main_Workflow/Hyperlink_extract/{DOC_CODE}_hyperlink_matches_with_clause.csv")

# ------------------------------------------------------------
# 1) 建 blocks
# ------------------------------------------------------------
def load_blocks(content_list_path: Path):
    with open(content_list_path, "r", encoding="utf-8") as f:
        raw = json.load(f)
    blocks = []
    for i, b in enumerate(raw):
        blocks.append({
            "block_idx": i,
            "type": b.get("type"),
            "page_idx": b.get("page_idx"),
            "bbox": b.get("bbox"),
            "text": (b.get("text") or b.get("table_body") or "").strip()
        })
    return blocks

# ------------------------------------------------------------
# 2) 條款行偵測
# ------------------------------------------------------------
HEADING_RE = re.compile(r'^\s*(\d+(?:\.\d+)*)[.\)]?\s+(.*\S)?\s*$')

def classify_heading(text: str) -> Optional[Tuple[str, int, str]]:
    """
    如果是條款/章節標題，回傳 (clause_id, level, title)
    - level = id 的段數，如 "1" -> 1, "1.1"->2, "1.1.1"->3
    否則回傳 None
    """
    m = HEADING_RE.match(text)
    if not m:
        return None
    cid = m.group(1)
    title = m.group(2) or ""
    level = cid.count(".") + 1
    return (cid, level, title.strip())

def find_headings(blocks):
    heads = []
    for b in blocks:
        if b["type"] != "text":
            continue
        h = classify_heading(b.get("text",""))
        if not h:
            continue
        cid, level, title = h
        heads.append({
            "block_idx": b["block_idx"],
            "page_idx": b["page_idx"],
            "clause_id": cid,
            "level": level,
            "title": title
        })
    heads.sort(key=lambda x: x["block_idx"])
    return heads

# ------------------------------------------------------------
# 3) 由 heading 建 spans + 條款全文
# ------------------------------------------------------------
def slice_body_text(blocks, start_idx, end_idx):
    texts = []
    for i in range(start_idx, end_idx):
        b = blocks[i]
        if b["type"] == "text":
            txt = b.get("text","").strip()
            if txt:
                texts.append(txt)
    return "\n".join(texts).strip()

def build_spans_dedup(blocks, heads):
    cands = []
    for i, h in enumerate(heads):
        start = h["block_idx"]
        end = heads[i+1]["block_idx"] if i+1 < len(heads) else len(blocks)
        body_len = len(slice_body_text(blocks, start+1, end))
        full_text = slice_body_text(blocks, start, end)   # ★ 新增：完整條款文字
        cands.append({
            "clause_id": h["clause_id"],
            "start_block": start,
            "end_block": end,
            "page_idx": h["page_idx"],
            "level": h["level"],
            "title": h["title"],
            "body_len": body_len,
            "full_text": full_text                        # ★ 新增
        })
    best = {}
    for c in cands:
        cid = c["clause_id"]
        cur = best.get(cid)
        if (cur is None) or (c["body_len"] > cur["body_len"]):
            best[cid] = c
    spans = sorted(best.values(), key=lambda x: x["start_block"])
    return spans


In [3]:
# ------------------------------------------------------------
# 4) 超連結 → block
# ------------------------------------------------------------
def parse_rect_string(rect_str: str):
    try:
        s = rect_str.strip()
        s = re.sub(r'^\s*Rect\(\s*', '', s, flags=re.I)
        s = re.sub(r'\)\s*$', '', s)
        return [float(c.strip()) for c in s.split(",")]
    except:
        return None

def normalize_rect(rect_coords, page_w, page_h):
    x0, y0, x1, y1 = rect_coords
    return [
        (x0 / page_w)  * 1000.0,
        (y0 / page_h)  * 1000.0,
        (x1 / page_w)  * 1000.0,
        (y1 / page_h)  * 1000.0,
    ]

def is_contained(inner_box, outer_box, tol=1.0):
    ix0, iy0, ix1, iy1 = inner_box
    ox0, oy0, ox1, oy1 = outer_box
    return (ox0 - tol) <= ix0 and (oy0 - tol) <= iy0 and (ox1 + tol) >= ix1 and (oy1 + tol) >= iy1

def build_page_index(blocks):
    page_map = defaultdict(list)
    for b in blocks:
        if b["page_idx"] is not None:
            page_map[b["page_idx"]].append(b)
    for pg in page_map:
        page_map[pg].sort(key=lambda bl: (bl["bbox"][1] if bl["bbox"] else 0,
                                          bl["bbox"][0] if bl["bbox"] else 0))
    return page_map

def nearest_text_block_idx(page_blocks, page0, link_bbox):
    best_idx, best_area = None, float("inf")
    for b in page_blocks.get(page0, []):
        if b["type"] != "text" or not b.get("bbox"):
            continue
        if is_contained(link_bbox, b["bbox"]):
            area = (b["bbox"][2]-b["bbox"][0]) * (b["bbox"][3]-b["bbox"][1])
            if area < best_area:
                best_area = area
                best_idx = b["block_idx"]
    return best_idx

# ------------------------------------------------------------
# 5) block_idx → clause
# ------------------------------------------------------------
def clause_of_block(spans, block_idx):
    if block_idx is None:
        return None, ""
    for sp in spans:
        if sp["start_block"] == block_idx or (sp["start_block"] < block_idx < sp["end_block"]):
            return sp["clause_id"], sp["full_text"]   # ★ 回傳文字
    return None, ""

def extract_doc_id_from_uri(uri: str) -> str:
    if uri.endswith(".pdf"):
        # 提取最後一段路徑，去掉 ".pdf"
        return uri.split("/")[-1].replace(".pdf", "")
    return ""


In [4]:
# ------------------------------------------------------------
# 6) 主流程
# ------------------------------------------------------------
def run_matching(content_list_path, links_csv_path, output_csv_path):
    blocks = load_blocks(content_list_path)
    heads  = find_headings(blocks)
    spans  = build_spans_dedup(blocks, heads)
    page_blocks = build_page_index(blocks)

    links_df = pd.read_csv(links_csv_path)
    out_rows = []

    for _, row in links_df.iterrows():
        page0 = int(row["page"])
        rect  = parse_rect_string(row["rect"])
        if not rect:
            continue
        link_bb = normalize_rect(rect, float(row["page_width"]), float(row["page_height"]))
        blk_idx = nearest_text_block_idx(page_blocks, page0, link_bb)
        cid, clause_text = clause_of_block(spans, blk_idx)

        uri = row["uri"]
        ref_doc_id = extract_doc_id_from_uri(uri)

        out_rows.append({
            "page": page0 + 1,
            "uri": row["uri"],
            "normalized_link_bbox": [round(c,2) for c in link_bb],
            "matched_block_idx": blk_idx,
            "clause_id": cid,
            "clause_text": clause_text,
            "referenced_doc_id": ref_doc_id
        })

    out_df = pd.DataFrame(out_rows)
    out_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ saved: {output_csv_path}  rows={len(out_df)}")
    return out_df

run_matching(CONTENT_LIST_JSON, INPUT_LINKS_CSV, OUTPUT_MATCH_CSV)

✅ saved: C:\Users\User\Desktop\GraphRAG\Main_Workflow\Hyperlink_extract\AML-1_hyperlink_matches_with_clause.csv  rows=6


,page,uri,normalized_link_bbox,matched_block_idx,clause_id,clause_text,referenced_doc_id
0,1,http://www.hkma.gov.hk/media/eng/doc/key-funct...,"[624.68, 224.95, 737.54, 241.35]",1.0,None,,IN
1,1,http://www.hkma.gov.hk/media/eng/doc/key-funct...,"[139.06, 241.35, 226.12, 257.75]",NaN,None,,GL
2,23,https://www.hkma.gov.hk/eng/key-functions/bank...,"[191.05, 439.64, 279.27, 470.3]",NaN,None,,
3,23,http://www.hkma.gov.hk/media/eng/doc/key-funct...,"[367.68, 439.64, 454.74, 470.3]",NaN,None,,GL
4,23,https://www.hkma.gov.hk/eng/index.shtml,"[543.92, 439.64, 605.23, 470.3]",NaN,None,,
5,23,http://www.hkma.gov.hk/media/eng/doc/key-funct...,"[700.02, 439.64, 812.89, 470.3]",NaN,None,,IN


# DownLoad PDF 

In [7]:
import pandas as pd
import requests
from pathlib import Path
from urllib.parse import urlparse
import os

# 增加這行來抑制 SSL 驗證的警告訊息
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

def download_pdfs_from_csv(csv_path: Path, download_dir: Path):
    """
    從 CSV 檔案中讀取 URI，並自動下載其中的 PDF 檔案。

    Args:
        csv_path (Path): 包含 'uri' 欄位的 CSV 檔案路徑。
        download_dir (Path): 用於儲存下載的 PDF 的資料夾路徑。
    """
    # 1. 確保下載資料夾存在
    download_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 下載資料夾已準備好: {download_dir}")

    # 2. 讀取 CSV 檔案
    try:
        df = pd.read_csv(csv_path)
        if 'uri' not in df.columns:
            print(f"❌ 錯誤: CSV 檔案 '{csv_path}' 中找不到 'uri' 欄位。")
            return
    except FileNotFoundError:
        print(f"❌ 錯誤: 找不到 CSV 檔案 '{csv_path}'。")
        return

    # 3. 獲取所有不重複的、以 .pdf 結尾的 URL
    urls_to_download = df['uri'].dropna().unique()
    pdf_urls = [url for url in urls_to_download if url.lower().endswith('.pdf')]
    
    if not pdf_urls:
        print("ℹ️ 在 CSV 中沒有找到任何有效的 PDF 連結。")
        return

    print(f"🔗 找到 {len(pdf_urls)} 個不重複的 PDF 連結準備下載。")
    
    # 4. 遍歷並下載每一個 PDF
    for url in pdf_urls:
        try:
            # 從 URL 中提取檔案名稱
            filename = os.path.basename(urlparse(url).path)
            if not filename: # 如果 URL 以 / 結尾，檔名可能為空
                filename = "downloaded_file.pdf"
            save_path = download_dir / filename

            # 檢查檔案是否已存在，避免重複下載
            if save_path.exists():
                print(f"⏭️  已跳過 (檔案已存在): {filename}")
                continue

            print(f"⏳ 正在下載: {url}")
            
            # 發送 GET 請求，設定超時並增加 verify=False 來忽略 SSL 錯誤
            response = requests.get(url, stream=True, timeout=20, verify=False)
            response.raise_for_status()  # 如果請求失敗 (如 404)，則會引發錯誤

            # 將內容寫入檔案
            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            print(f"✅ 下載成功: {filename}")

        except requests.exceptions.RequestException as e:
            print(f"❌ 下載失敗: {url} - 錯誤: {e}")
        except Exception as e:
            print(f"❌ 處理失敗: {url} - 發生未知錯誤: {e}")

    print("\n--- 所有下載任務已完成 ---")


# --- 主執行區塊 ---

# --- 請在這裡配置你的輸入和輸出路徑 ---
DOC_CODE = "AML-1"

# 輸入的 CSV 檔案路徑 (請確保這個檔案存在)
INPUT_MATCHES_CSV = Path(f"c:/Users/User/Desktop/GraphRAG/Main_Workflow/Hyperlink_extract/{DOC_CODE}.links_normalized.csv")

# 你希望將 PDF 下載到哪個資料夾
PDF_DOWNLOAD_DIR = Path(f"c:/Users/User/Desktop/GraphRAG/Main_Workflow/downloaded_pdfs/{DOC_CODE}")

# 執行下載函數
# 在運行前，請確保你已經安裝了 requests 函式庫 (pip install requests)
download_pdfs_from_csv(
    csv_path=INPUT_MATCHES_CSV,
    download_dir=PDF_DOWNLOAD_DIR
)

📁 下載資料夾已準備好: c:\Users\User\Desktop\GraphRAG\Main_Workflow\downloaded_pdfs\AML-1
🔗 找到 2 個不重複的 PDF 連結準備下載。
⏳ 正在下載: http://www.hkma.gov.hk/media/eng/doc/key-functions/banking-stability/supervisory-policy-manual/IN.pdf
✅ 下載成功: IN.pdf
⏳ 正在下載: http://www.hkma.gov.hk/media/eng/doc/key-functions/banking-stability/supervisory-policy-manual/GL.pdf
✅ 下載成功: GL.pdf

--- 所有下載任務已完成 ---
